## Frame the Problem and Big Picture

**Client Background:**
- Our client, FinSight Analytics, is a financial risk assessment consultancy that provides services to major investment firms, banks, and private equity companies across North America. They've approached Team Gort to develop a more effective bankruptcy prediction model that their clients can use to assess investment and lending risks.

**1. Define the objective in business terms:**
- The objective for this machine learning model is to accurately predict whether a company will declare bankruptcy within the next two years based on its financial ratios and indicators.

**2. How will your solution be used?**
- This model will be used by financial institutions (banks, investment firms, and creditors) to assess the bankruptcy risk of companies they're considering investing in or extending credit to. It will serve as an early warning system, allowing these stakeholders to make more informed decisions, adjust lending terms based on risk profiles, implement early intervention strategies, and optimize their portfolio management.

**3. What are the current solutions/workarounds (if any)?**
- Currently, financial institutions rely on traditional credit scoring models and manual analysis by financial analysts. These approaches often fail to detect early warning signs of bankruptcy and tend to over-rely on limited metrics like payment history and debt ratios while overlooking other critical financial indicators.

**4. How should you frame this problem?**
- This is a supervised binary classification problem since we are trying to predict whether a company will go bankrupt (1) or not (0) based on historical financial data. This would be a batch learning solution that gets periodically updated as new financial data becomes available, though predictions would be generated whenever a new company needs to be evaluated.

**5. How should performance be measured? Is the performance measure aligned with the business objective?**
- Our objective is to be able to predict at least 80% of the companies that will go bankrupt while maintaining a false positive rate below 15%. This aligns with our business objective since missing a bankruptcy prediction can result in significant financial losses, while incorrectly flagging healthy companies can lead to missed business opportunities.

**6. What would be the minimum performance needed to reach the business objective?**
- The minimum performance needed would be the ability to correctly identify at least 80% of companies that will go bankrupt, with a false positive rate below 15%, and provide predictions at least 12 months before bankruptcy occurs.

**7. What are comparable problems? Can you reuse experience or tools?**
- This problem is comparable to other financial risk assessment problems such as credit default prediction and loan delinquency prediction. We can leverage our experience with classification algorithms like Random Forest, Gradient Boosting, and logistic regression, which have proven effective in similar financial prediction tasks. We can also use techniques from previous projects dealing with imbalanced datasets.

**8. Is human expertise available?**
- Yes, our client has experience with financial risk assessment and bankruptcy prediction and has provided us with good insight on key financial indicators that typically precede bankruptcy. They also have analysts who can help interpret complex financial ratios.

**9. How would you solve the problem manually?**
- To solve this problem manually, we would need to analyze the historical financial data of companies that went bankrupt and those that remained solvent. We would look for patterns in financial ratios, identify negative trends in profitability and liquidity, examine cash flow patterns, assess debt structure, and combine these factors to make a risk assessment. We would also need to consider industry-specific benchmarks since financial ratios can vary significantly across different sectors.

**10. List the assumptions you (or others) have made so far. Verify assumptions if possible.**
- We have made the assumption that financial ratios and indicators from company reports are strong predictors of future bankruptcy. We also assume that the relationship between these indicators and bankruptcy risk remains relatively stable over time. Another assumption is that the provided dataset contains sufficient examples of both bankrupt and non-bankrupt companies to train an effective model. We will need to verify these assumptions during our exploratory data analysis phase.
